- How To Create Tools
- How to use built-in tools and toolkits
- How to use chat models to call tools
- How to pass tool outputs to chat models

In [1]:
from langchain_groq import ChatGroq
import os
from langchain_core.tools import tool


/Users/manasvi/Github/Agents-Langchain-Langraph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### creating tool

In [2]:
@tool
def mathematics(a: float, b: float) -> float:
    """Add, Subtract, Multiply, or Divide two numbers"""
    return a + b, a-b , a*b , a/b



print(mathematics.name)
print(mathematics.description)
print(mathematics.args)

mathematics
Add, Subtract, Multiply, or Divide two numbers
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


### there are lot of other ways of tools as well async, annotated(list), structured tool

In [3]:
mathematics.invoke({"a": 1, "b": 2})

(3.0, -1.0, 2.0, 0.5)

#### laoding llm

In [4]:
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain.chat_models import init_chat_model


llm = init_chat_model("groq:llama-3.1-8b-instant")

#### creating agent

In [5]:
# create an agent from the tool
from langchain.agents import create_agent
tools = [mathematics]
prompt = (
    """
You are a math assistant.

When the user gives you two numbers:
1. Parse the numbers from the user message.
2. Add 2 to each number to get new_a and new_b.
3. Call the `mathematics` tool ONCE with a=new_a and b=new_b.
4. Then explain to the user:
   - The new numbers
   - The sum, difference, product, and quotient from the tool. """
)
agent = create_agent(llm, tools, system_prompt=prompt)



In [6]:
query  = " Numbers are 4 and 2"
agent.invoke({"input": query})

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

 Numbers are 4 and 2
================================== Ai Message ==================================
Tool Calls:
  mathematics (xpbvsegsc)
 Call ID: xpbvsegsc
  Args:
    a: 6
    b: 4
================================= Tool Message =================================
Name: mathematics

[10.0, 2.0, 24.0, 1.5]
================================== Ai Message ==================================

The new numbers are 6 and 4.

The sum of the new numbers is 10.
The difference between the new numbers is 2.
The product of the new numbers is 24.
The quotient of the new numbers is 1.5.


#### using wikipedia pre build tool

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query":"langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors


In [19]:
### creating wikipedia a tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(wiki_tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.




In [26]:
@tool
def mathematics(a: float, b: float) -> float:
    """Add, Subtract, Multiply, or Divide two numbers"""
    return a + b, a-b , a*b , a/b



import requests
from langchain_core.tools import tool

@tool
def weather(city: str):
    """Get current weather-related data for a given city using the Open-Meteo geocoding API."""
    return requests.get(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
    ).json()




In [23]:
tools = [weather, mathematics, wiki_tool]

In [25]:
from langchain.agents import create_agent

prompt = """
You are a helpful assistant.
Answer the user's question based on the information provided.

"""

agent = create_agent(
    llm,
    tools,
    system_prompt=prompt,
)


query  = "what is langchain"
agent.invoke({"input": query})

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()



================================ Human Message =================================

what is langchain
================================== Ai Message ==================================
Tool Calls:
  wikipedia (q08pfzap4)
 Call ID: q08pfzap4
  Args:
    query: Langchain AI
================================= Tool Message =================================
Name: wikipedia

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.


================================== Ai Message ==================================
Tool Calls:
  wikipedia (rcjhm9etc)
 Call ID: rcjhm9etc
  Args:
    query: Langchain AI use cases
================================= Tool Message =================================
Name: wikipedia

Page: L

### chain

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate

# Create a more complex chain
def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])
    
    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])
    
    # Build the chain - Method 1: Sequential execution
    story_chain = (
        story_prompt 
        | llm 
        | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | llm
        | StrOutputParser()
    )
    return analysis_chain

In [28]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1589a6990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x158c97200>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables={}, messages=

In [32]:
result = chain.invoke({
    "theme": "Life of a boy",
    "character": "Ram : A boy from school",
    "setting": "village life"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
**Analysis of "The Boy with a Golden Smile"**

"The Boy with a Golden Smile" is a heartwarming and uplifting story that embodies the spirit of childhood innocence, kindness, and wonder. On the surface, it appears to be a simple tale of a young boy's adventures in a quaint village, but upon closer examination, it reveals itself to be a rich and nuanced exploration of the human experience.

**Themes**

One of the primary themes of the story is the importance of empathy and compassion. Ram's kindness towards the wounded bird, Kali, sets off a chain of events that not only helps the bird recover but also brings the community together. This theme is reinforced by the villagers' amazement and admiration for Ram's selflessness and gentle nature. The story suggests that even small acts of kindness can have a profound impact on those around us.

Another significant theme is the power of curiosity and exploration. Ram's love for nature and animals drives him to discover new s